In [ ]:
import sqlite3
from datetime import datetime

def create_database():
    connection = sqlite3.connect('course_management_system.db')
    cursor = connection.cursor()

    # Create tables
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Student (
            sid INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            email TEXT UNIQUE NOT NULL,
            gpa REAL CHECK(gpa BETWEEN 0 AND 4),
            age INTEGER CHECK(age >= 0),
            grade INTEGER CHECK(grade >= 0),
            credits_completed INTEGER CHECK(credits_completed >= 0),
            password TEXT NOT NULL
        )
    ''')

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Professor (
            pid INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            department TEXT NOT NULL
        )
    ''')

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Course (
            cid INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            credits INTEGER CHECK(credits > 0),
            description TEXT,
            capacity INTEGER CHECK(capacity <= 25),
            professor_id INTEGER,
            FOREIGN KEY (professor_id) REFERENCES Professor(pid)
        )
    ''')

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Prerequisite (
            course_id INTEGER,
            prereq_id INTEGER,
            PRIMARY KEY (course_id, prereq_id),
            FOREIGN KEY (course_id) REFERENCES Course(cid),
            FOREIGN KEY (prereq_id) REFERENCES Course(cid)
        )
    ''')

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Schedule (
            course_id INTEGER,
            day_of_week TEXT NOT NULL,
            start_time TEXT CHECK(start_time >= '08:00' AND start_time <= '22:00'),
            end_time TEXT CHECK(end_time >= '08:00' AND end_time <= '22:00'),
            location TEXT NOT NULL,
            FOREIGN KEY (course_id) REFERENCES Course(cid)
        )
    ''')

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Enrollment (
            enrollment_id INTEGER PRIMARY KEY AUTOINCREMENT,
            student_id INTEGER,
            course_id INTEGER,
            FOREIGN KEY (student_id) REFERENCES Student(sid),
            FOREIGN KEY (course_id) REFERENCES Course(cid)
        )
    ''')

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS TeachingAssignment (
            professor_id INTEGER,
            course_id INTEGER,
            PRIMARY KEY (professor_id, course_id),
            FOREIGN KEY (professor_id) REFERENCES Professor(pid),
            FOREIGN KEY (course_id) REFERENCES Course(cid)
        )
    ''')

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Enrollment (
          enrollment_id INTEGER PRIMARY KEY AUTOINCREMENT,
          student_id INTEGER,
          course_id INTEGER,
          FOREIGN KEY (student_id) REFERENCES Student(sid),
          FOREIGN KEY (course_id) REFERENCES Course(cid)
        )
    ''')

    connection.commit()
    connection.close()


In [ ]:
def insert_sample_data():
    connection = sqlite3.connect('course_management_system.db')
    cursor = connection.cursor()

    # Insert Professors
    professors = [
        (1, 'Dr. Michelle Zhu', 'Computer Science'),
        (2, 'Dr. Bogdan Nita', 'Mathematics'),
        (3, 'Dr. Carlos Monila', 'Biology'),
        (4, 'Dr. John Doe', 'Physics'),
        (5, 'Professor Micheal Allen', 'Anthropology')
    ]
    cursor.executemany('INSERT OR IGNORE INTO Professor (pid, name, department) VALUES (?, ?, ?)', professors)

    # Insert Courses
    courses = [
        (1, 'Intro to CS', 3, 'Introduction to programming', 25, 1),
        (2, 'Data Structures', 4, 'Organization of data', 25, 1),
        (3, 'Calculus I', 4, 'Basics of calculus', 20, 2),
        (4, 'Biology I', 3, 'Basics of Biology', 25, 3),
        (5, 'Algorithms', 3, 'Advanced algorithms', 25, 1),
        (6, 'Physics I', 4, 'Fundamentals of Physics', 20, 4),
        (7, 'Discrete Math', 3, 'Mathematical foundations for CS', 25, 2),
        (8, 'Anthropology Basics', 3, 'Introduction to Anthropology', 30, 5),
        (9, 'Linear Algebra', 4, 'Matrix theory and applications', 25, 2),
        (10, 'Biology II', 4, 'Advanced Biology concepts', 25, 3)
    ]
    cursor.executemany('INSERT OR IGNORE INTO Course (cid, name, credits, description, capacity, professor_id) VALUES (?, ?, ?, ?, ?, ?)', courses)

    # Insert Prerequisites
    prerequisites = [
        (2, 1),  # Data Structures requires Intro to CS
        (5, 2),  # Algorithms requires Data Structures
        (10, 4)  # Biology II requires Biology I
    ]
    cursor.executemany('INSERT OR IGNORE INTO Prerequisite (course_id, prereq_id) VALUES (?, ?)', prerequisites)

    # Insert Students
    students = [
        (1, 'Parth Patel', 'patelp24@montclair.edu', 3.8, 20, 2, 60, 'password1'),
        (2, 'Krina Parekh', 'parekhh@montclair.edu', 3.2, 21, 3, 90, 'password2'),
        (3, 'Anika Sharma', 'sharmaa@montclair.edu', 3.6, 19, 1, 30, 'password3'),
        (4, 'Raj Mehta', 'mehtar@montclair.edu', 3.1, 22, 4, 100, 'password4'),
        (5, 'Maya Gupta', 'guptam@montclair.edu', 3.9, 21, 3, 90, 'password5'),
        (6, 'Ishaan Bhatt', 'bhatti@montclair.edu', 3.3, 20, 2, 60, 'password6'),
        (7, 'Sia Patel', 'patels@montclair.edu', 3.5, 19, 2, 70, 'password7'),
        (8, 'Kunal Shah', 'shahk@montclair.edu', 3.0, 23, 4, 110, 'password8'),
        (9, 'Riya Desai', 'desair@montclair.edu', 3.7, 20, 3, 80, 'password9'),
        (10, 'Arjun Rana', 'ranaa@montclair.edu', 3.4, 22, 4, 105, 'password10')
    ]
    cursor.executemany('INSERT OR IGNORE INTO Student (sid, name, email, gpa, age, grade, credits_completed, password) VALUES (?, ?, ?, ?, ?, ?, ?, ?)', students)

    # Insert Schedules with time conflicts
    schedules = [
        (1, 'Monday', '10:00', '11:30', 'Room 101'),
        (2, 'Tuesday', '14:00', '15:30', 'Room 202'),
        (3, 'Wednesday', '09:00', '10:30', 'Room 303'),
        (4, 'Thursday', '11:00', '12:30', 'Room 404'),
        (5, 'Tuesday', '14:00', '15:30', 'Room 505'),  # Time conflict with Data Structures
        (6, 'Friday', '13:00', '14:30', 'Room 606'),
        (7, 'Monday', '11:30', '13:00', 'Room 707'),
        (8, 'Thursday', '14:00', '15:30', 'Room 808'),
        (9, 'Wednesday', '10:30', '12:00', 'Room 909'),
        (10, 'Friday', '14:30', '16:00', 'Room 1010')
    ]
    cursor.executemany('INSERT OR IGNORE INTO Schedule (course_id, day_of_week, start_time, end_time, location) VALUES (?, ?, ?, ?, ?)', schedules)

    # Teaching Assignment Table
    teaching_assignments = [
        (1, 1),  # Dr. Michelle Zhu teaches Intro to CS
        (1, 2),  # Dr. Michelle Zhu teaches Data Structures
        (1, 5),  # Dr. Michelle Zhu teaches Algorithms
        (2, 3),  # Dr. Bogdan Nita teaches Calculus I
        (2, 7),  # Dr. Bogdan Nita teaches Discrete Math
        (2, 9),  # Dr. Bogdan Nita teaches Linear Algebra
        (3, 4),  # Dr. Carlos Monila teaches Biology I
        (3, 10), # Dr. Carlos Monila teaches Biology II
        (4, 6),  # Dr. John Doe teaches Physics I
        (5, 8)   # Professor Micheal Allen teaches Anthropology Basics
    ]
    cursor.executemany('INSERT OR IGNORE INTO TeachingAssignment (professor_id, course_id) VALUES (?, ?)', teaching_assignments)

    # Insert Enrollments
    enrollments = [
        (1, 1),  # Parth Patel -> Intro to CS
        (1, 2),  # Parth Patel -> Data Structures
        (2, 3),  # Krina Parekh -> Calculus I
        (2, 5),  # Krina Parekh -> Algorithms
        (3, 1),  # Anika Sharma -> Intro to CS
        (3, 4),  # Anika Sharma -> Biology I
        (4, 6),  # Raj Mehta -> Physics I
        (4, 3),  # Raj Mehta -> Calculus I
        (5, 4),  # Maya Gupta -> Biology I
        (5, 10), # Maya Gupta -> Biology II
        (6, 7),  # Ishaan Bhatt -> Discrete Math
        (6, 2),  # Ishaan Bhatt -> Data Structures
        (7, 1),  # Sia Patel -> Intro to CS
        (7, 5),  # Sia Patel -> Algorithms
        (8, 9),  # Kunal Shah -> Linear Algebra
        (8, 6),  # Kunal Shah -> Physics I
        (9, 3),  # Riya Desai -> Calculus I
        (9, 4),  # Riya Desai -> Biology I
        (10, 8), # Arjun Rana -> Anthropology Basics
        (10, 10) # Arjun Rana -> Biology II
    ]
    cursor.executemany('INSERT OR IGNORE INTO Enrollment (student_id, course_id) VALUES (?, ?)', enrollments)


    connection.commit()
    connection.close()

In [ ]:
def list_courses():
    connection = sqlite3.connect('course_management_system.db')
    cursor = connection.cursor()

    cursor.execute('''
        SELECT c.cid, c.name, c.credits, c.description, c.capacity, p.name, s.day_of_week, s.start_time, s.end_time, s.location
        FROM Course c
        LEFT JOIN Professor p ON c.professor_id = p.pid
        LEFT JOIN Schedule s ON c.cid = s.course_id
    ''')
    courses = cursor.fetchall()

    print("\nCourses Available:")
    for course in courses:
        print(f"Course ID: {course[0]}, Name: {course[1]}, Credits: {course[2]}, Description: {course[3]}, Capacity: {course[4]}")
        print(f"  Professor: {course[5]}, Day: {course[6]}, Time: {course[7]} - {course[8]}, Location: {course[9]}")
    connection.close()


In [ ]:
def enroll_student(student_id):
    course_id = int(input("Enter Course ID: ").strip())
    connection = sqlite3.connect('course_management_system.db')
    cursor = connection.cursor()

    # Check if student is already enrolled in the course
    cursor.execute('SELECT * FROM Enrollment WHERE student_id = ? AND course_id = ?', (student_id, course_id))
    if cursor.fetchone():
        connection.close()
        return "Already enrolled in the class."

    # Check for schedule conflicts
    cursor.execute('''
        SELECT s1.course_id, c1.name, s1.day_of_week, s1.start_time, s1.end_time
        FROM Schedule s1
        JOIN Course c1 ON s1.course_id = c1.cid
        WHERE s1.course_id = ?
    ''', (course_id,))
    new_course_schedule = cursor.fetchone()

    cursor.execute('''
        SELECT s2.course_id, c2.name, s2.day_of_week, s2.start_time, s2.end_time
        FROM Enrollment e
        JOIN Schedule s2 ON e.course_id = s2.course_id
        JOIN Course c2 ON s2.course_id = c2.cid
        WHERE e.student_id = ?
    ''', (student_id,))
    enrolled_schedules = cursor.fetchall()

    for es in enrolled_schedules:
        if es[2] == new_course_schedule[2] and not (es[4] <= new_course_schedule[3] or es[3] >= new_course_schedule[4]):
            connection.close()
            return f"Time conflict with {es[1]}."

    # Check prerequisites
    cursor.execute('SELECT prereq_id FROM Prerequisite WHERE course_id = ?', (course_id,))
    prerequisites = cursor.fetchall()

    for prereq in prerequisites:
        cursor.execute('SELECT * FROM Enrollment WHERE student_id = ? AND course_id = ?', (student_id, prereq[0]))
        if not cursor.fetchone():
            connection.close()
            return f"Prerequisite Course ID {prereq[0]} not completed."

    # Check course capacity
    cursor.execute('''
        SELECT c.capacity, COUNT(e.course_id)
        FROM Course c
        LEFT JOIN Enrollment e ON c.cid = e.course_id
        WHERE c.cid = ?
    ''', (course_id,))
    capacity, enrolled = cursor.fetchone()

    if enrolled >= capacity:
        connection.close()
        return "Course is full."

    # Enroll the student
    cursor.execute('INSERT INTO Enrollment (student_id, course_id) VALUES (?, ?)', (student_id, course_id))
    connection.commit()
    connection.close()
    return "Enrolled successfully."


In [ ]:
def withdraw_student(student_id):
    course_id = int(input("Enter Course ID to withdraw from: ").strip())
    connection = sqlite3.connect('course_management_system.db')
    cursor = connection.cursor()

    # Check if the student is enrolled in the course
    cursor.execute('SELECT * FROM Enrollment WHERE student_id = ? AND course_id = ?', (student_id, course_id))
    if not cursor.fetchone():
        connection.close()
        return "You are not enrolled in this course."

    # Perform withdrawal
    cursor.execute('DELETE FROM Enrollment WHERE student_id = ? AND course_id = ?', (student_id, course_id))
    connection.commit()
    connection.close()
    return "Successfully withdrawn from the course."

In [ ]:
def search_courses():
    substring = input("Enter course name substring to search: ").strip()
    connection = sqlite3.connect('course_management_system.db')
    cursor = connection.cursor()

    cursor.execute('''
        SELECT c.cid, c.name, c.credits, c.description, c.capacity, p.name
        FROM Course c
        LEFT JOIN Professor p ON c.professor_id = p.pid
        WHERE c.name LIKE ?
    ''', (f"%{substring}%",))
    results = cursor.fetchall()

    print("\nSearch Results:")
    for course in results:
        print(f"Course ID: {course[0]}, Name: {course[1]}, Credits: {course[2]}, Description: {course[3]}, Capacity: {course[4]}, Professor: {course[5]}")
    connection.close()


In [ ]:
def view_enrolled_classes(student_id):
    connection = sqlite3.connect('course_management_system.db')
    cursor = connection.cursor()

    cursor.execute('''
        SELECT DISTINCT c.cid, c.name, c.credits, s.day_of_week, s.start_time, s.end_time, p.name
        FROM Enrollment e
        JOIN Course c ON e.course_id = c.cid
        JOIN Schedule s ON c.cid = s.course_id
        LEFT JOIN Professor p ON c.professor_id = p.pid
        WHERE e.student_id = ?
    ''', (student_id,))
    enrolled_courses = cursor.fetchall()

    print("\nMy Classes:")
    total_credits = 0
    for course in enrolled_courses:
        total_credits += course[2]
        print(f"Course ID: {course[0]}, Name: {course[1]}, Credits: {course[2]}")
        print(f"  Day: {course[3]}, Time: {course[4]} - {course[5]}, Professor: {course[6]}")
    print(f"Total Credits: {total_credits}")
    connection.close()

In [ ]:
def view_prerequisites():
    course_id = int(input("Enter Course ID to view prerequisites: ").strip())
    connection = sqlite3.connect('course_management_system.db')
    cursor = connection.cursor()

    cursor.execute('SELECT prereq_id FROM Prerequisite WHERE course_id = ?', (course_id,))
    prerequisites = cursor.fetchall()

    if prerequisites:
        print("Prerequisites for this course:", [pr[0] for pr in prerequisites])
    else:
        print("No prerequisites for this course.")
    connection.close()


In [ ]:
def list_professors():
    connection = sqlite3.connect('course_management_system.db')
    cursor = connection.cursor()

    cursor.execute('''
        SELECT p.pid, p.name, p.department, GROUP_CONCAT(c.name, ', ') AS courses
        FROM Professor p
        LEFT JOIN Course c ON p.pid = c.professor_id
        GROUP BY p.pid
    ''')
    professors = cursor.fetchall()

    print("\nProfessors:")
    for professor in professors:
        print(f"ID: {professor[0]}, Name: {professor[1]}, Department: {professor[2]}, Courses: {professor[3]}")
    connection.close()


In [ ]:
def list_students():
    connection = sqlite3.connect('course_management_system.db')
    cursor = connection.cursor()

    cursor.execute('SELECT sid, name, email, gpa, age, grade, credits_completed FROM Student')
    students = cursor.fetchall()

    if students:
        print("\nList of Students:")
        print(f"{'ID':<5} {'Name':<20} {'Email':<30} {'GPA':<5} {'Age':<5} {'Grade':<5} {'Credits Completed':<5}")
        print("-" * 80)
        for student in students:
            print(f"{student[0]:<5} {student[1]:<20} {student[2]:<30} {student[3]:<5.2f} {student[4]:<5} {student[5]:<5} {student[6]:<5}")
    else:
        print("No students found in the database.")

    connection.close()

In [ ]:
def add_course():
    connection = sqlite3.connect('course_management_system.db')
    cursor = connection.cursor()

    name = input("Enter course name: ").strip()
    credits = int(input("Enter number of credits: ").strip())
    description = input("Enter description: ").strip()
    capacity = int(input("Enter capacity (max 25): ").strip())
    professor_id = int(input("Enter Professor ID: ").strip())

    cursor.execute('''
        INSERT INTO Course (name, credits, description, capacity, professor_id)
        VALUES (?, ?, ?, ?, ?)
    ''', (name, credits, description, capacity, professor_id))
    connection.commit()
    print("Course added successfully.")
    connection.close()

def edit_course():
    connection = sqlite3.connect('course_management_system.db')
    cursor = connection.cursor()

    cid = int(input("Enter Course ID to edit: ").strip())
    cursor.execute('SELECT * FROM Course WHERE cid = ?', (cid,))
    course = cursor.fetchone()
    if not course:
        print("Course not found.")
        connection.close()
        return

    name = input(f"Enter new name (current: {course[1]}): ").strip() or course[1]
    credits = input(f"Enter new credits (current: {course[2]}): ").strip() or course[2]
    description = input(f"Enter new description (current: {course[3]}): ").strip() or course[3]
    capacity = input(f"Enter new capacity (current: {course[4]}): ").strip() or course[4]
    professor_id = input(f"Enter new Professor ID (current: {course[5]}): ").strip() or course[5]

    cursor.execute('''
        UPDATE Course SET name = ?, credits = ?, description = ?, capacity = ?, professor_id = ?
        WHERE cid = ?
    ''', (name, credits, description, capacity, professor_id, cid))
    connection.commit()
    print("Course updated successfully.")
    connection.close()

def delete_course():
    connection = sqlite3.connect('course_management_system.db')
    cursor = connection.cursor()

    cid = int(input("Enter Course ID to delete: ").strip())
    cursor.execute('DELETE FROM Course WHERE cid = ?', (cid,))
    connection.commit()
    print("Course deleted successfully.")
    connection.close()


In [ ]:
def add_student():
    connection = sqlite3.connect('course_management_system.db')
    cursor = connection.cursor()

    name = input("Enter name: ").strip()
    email = input("Enter email: ").strip()

    # GPA Validation
    gpa = float(input("Enter GPA (0.0 - 4.0): ").strip())
    if gpa < 0.0 or gpa > 4.0:
        return "GPA must be between 0.0 and 4.0."

    # Age Validation
    age = int(input("Enter age (must be positive): ").strip())
    if age < 0:
        return "Age must be a positive value."

    # Grade Validation
    grade = int(input("Enter grade (must be positive): ").strip())
    if grade < 0:
        return "Grade must be a positive value."

    # Credits Validation
    credits_completed = int(input("Enter credits completed (must be positive): ").strip())
    if credits_completed < 0:
        return "Credits completed must be a positive value."

    password = input("Enter password: ").strip()

    cursor.execute('''
        INSERT INTO Student (name, email, gpa, age, grade, credits_completed, password)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', (name, email, gpa, age, grade, credits_completed, password))
    connection.commit()
    connection.close()
    return "Student added successfully."

def edit_student():
    connection = sqlite3.connect('course_management_system.db')
    cursor = connection.cursor()

    sid = int(input("Enter Student ID to edit: ").strip())
    cursor.execute('SELECT * FROM Student WHERE sid = ?', (sid,))
    student = cursor.fetchone()
    if not student:
        print("Student not found.")
        connection.close()
        return

    name = input(f"Enter new name (current: {student[1]}): ").strip() or student[1]
    email = input(f"Enter new email (current: {student[2]}): ").strip() or student[2]
    gpa = input(f"Enter new GPA (current: {student[3]}): ").strip() or student[3]
    age = input(f"Enter new age (current: {student[4]}): ").strip() or student[4]
    grade = input(f"Enter new grade (current: {student[5]}): ").strip() or student[5]
    credits_completed = input(f"Enter new credits completed (current: {student[6]}): ").strip() or student[6]

    cursor.execute('''
        UPDATE Student SET name = ?, email = ?, gpa = ?, age = ?, grade = ?, credits_completed = ?
        WHERE sid = ?
    ''', (name, email, gpa, age, grade, credits_completed, sid))
    connection.commit()
    print("Student updated successfully.")
    connection.close()

def delete_student():
    connection = sqlite3.connect('course_management_system.db')
    cursor = connection.cursor()

    sid = int(input("Enter Student ID to delete: ").strip())
    cursor.execute('DELETE FROM Student WHERE sid = ?', (sid,))
    connection.commit()
    print("Student deleted successfully.")
    connection.close()


In [ ]:
def add_professor():
    connection = sqlite3.connect('course_management_system.db')
    cursor = connection.cursor()

    name = input("Enter professor's name: ").strip()
    department = input("Enter department: ").strip()

    cursor.execute('''
        INSERT INTO Professor (name, department)
        VALUES (?, ?)
    ''', (name, department))
    connection.commit()
    print("Professor added successfully.")
    connection.close()

def edit_professor():
    connection = sqlite3.connect('course_management_system.db')
    cursor = connection.cursor()

    pid = int(input("Enter Professor ID to edit: ").strip())
    cursor.execute('SELECT * FROM Professor WHERE pid = ?', (pid,))
    professor = cursor.fetchone()
    if not professor:
        print("Professor not found.")
        connection.close()
        return

    name = input(f"Enter new name (current: {professor[1]}): ").strip() or professor[1]
    department = input(f"Enter new department (current: {professor[2]}): ").strip() or professor[2]

    cursor.execute('''
        UPDATE Professor SET name = ?, department = ?
        WHERE pid = ?
    ''', (name, department, pid))
    connection.commit()
    print("Professor updated successfully.")
    connection.close()

def delete_professor():
    connection = sqlite3.connect('course_management_system.db')
    cursor = connection.cursor()

    pid = int(input("Enter Professor ID to delete: ").strip())
    cursor.execute('DELETE FROM Professor WHERE pid = ?', (pid,))
    connection.commit()
    print("Professor deleted successfully.")
    connection.close()


In [ ]:
def list_enrolled_students(course_id):
    connection = sqlite3.connect('course_management_system.db')
    cursor = connection.cursor()

    cursor.execute('''
        SELECT s.sid, s.name, s.email, s.gpa, s.credits_completed
        FROM Enrollment e
        JOIN Student s ON e.student_id = s.sid
        WHERE e.course_id = ?
    ''', (course_id,))
    students = cursor.fetchall()

    if students:
        print(f"\nStudents enrolled in Course ID {course_id}:")
        for student in students:
            print(f"ID: {student[0]}, Name: {student[1]}, Email: {student[2]}, GPA: {student[3]}, Credits Completed: {student[4]}")
    else:
        print(f"No students are currently enrolled in Course ID {course_id}.")

    connection.close()


In [ ]:
def check_professor_workload():
    connection = sqlite3.connect('course_management_system.db')
    cursor = connection.cursor()

    # List all professors for user selection
    cursor.execute('SELECT pid, name, department FROM Professor')
    professors = cursor.fetchall()

    print("\nProfessors:")
    for professor in professors:
        print(f"ID: {professor[0]}, Name: {professor[1]}, Department: {professor[2]}")

    professor_id = int(input("\nEnter Professor ID to view their workload: ").strip())

    # Get courses and workload for the selected professor
    cursor.execute('''
        SELECT c.cid, c.name, c.credits
        FROM Course c
        WHERE c.professor_id = ?
    ''', (professor_id,))
    courses = cursor.fetchall()

    if courses:
        print(f"\nCourses taught by Professor ID {professor_id}:")
        total_credits = 0
        for course in courses:
            total_credits += course[2]
            print(f"Course ID: {course[0]}, Name: {course[1]}, Credits: {course[2]}")
        print(f"Total Workload (Credits): {total_credits}")
    else:
        print(f"Professor ID {professor_id} is not teaching any courses.")

    connection.close()


In [ ]:
def assign_professor_to_course():
    connection = sqlite3.connect('course_management_system.db')
    cursor = connection.cursor()

    # List all professors
    cursor.execute('SELECT pid, name, department FROM Professor')
    professors = cursor.fetchall()

    print("\nProfessors:")
    for professor in professors:
        print(f"ID: {professor[0]}, Name: {professor[1]}, Department: {professor[2]}")

    professor_id = int(input("\nEnter Professor ID to assign: ").strip())

    # Check if the professor exists
    cursor.execute('SELECT * FROM Professor WHERE pid = ?', (professor_id,))
    professor = cursor.fetchone()
    if not professor:
        print("Invalid Professor ID. Returning to menu.")
        connection.close()
        return

    # List all courses
    cursor.execute('SELECT cid, name, professor_id FROM Course')
    courses = cursor.fetchall()

    print("\nCourses:")
    for course in courses:
        assigned_professor = "None" if not course[2] else f"Professor ID {course[2]}"
        print(f"Course ID: {course[0]}, Name: {course[1]}, Assigned Professor: {assigned_professor}")

    course_id = int(input("\nEnter Course ID to assign to the professor: ").strip())

    # Check if the course exists
    cursor.execute('SELECT * FROM Course WHERE cid = ?', (course_id,))
    course = cursor.fetchone()
    if not course:
        print("Invalid Course ID. Returning to menu.")
        connection.close()
        return

    # Assign the professor to the course
    cursor.execute('''
        UPDATE Course
        SET professor_id = ?
        WHERE cid = ?
    ''', (professor_id, course_id))
    connection.commit()

    print(f"Professor ID {professor_id} has been assigned to Course ID {course_id}.")
    connection.close()

In [ ]:
def student_menu():
    connection = sqlite3.connect('course_management_system.db')
    cursor = connection.cursor()

    # Request Student ID
    student_id = int(input("Enter your Student ID (-1 to create a new student): ").strip())
    if student_id == -1:
        # Create a new student
        name = input("Enter your name: ").strip()
        email = input("Enter your email: ").strip()
        gpa = float(input("Enter your GPA (0.0 - 4.0): ").strip())
        if gpa < 0.0 or gpa > 4.0:
            print("GPA must be between 0.0 and 4.0.")
            return
        age = int(input("Enter your age: ").strip())
        if age < 0:
            print("Age must be a positive number.")
            return
        grade = int(input("Enter your grade: ").strip())
        if grade < 0:
            print("Grade must be a positive number.")
            return
        credits_completed = int(input("Enter credits completed: ").strip())
        if credits_completed < 0:
            print("Credits completed must be a positive number.")
            return
        password = input("Create a password: ").strip()

        cursor.execute('''
            INSERT INTO Student (name, email, gpa, age, grade, credits_completed, password)
            VALUES (?, ?, ?, ?, ?, ?, ?)
        ''', (name, email, gpa, age, grade, credits_completed, password))
        connection.commit()

        cursor.execute('SELECT sid FROM Student WHERE email = ?', (email,))
        student_id = cursor.fetchone()[0]
        print(f"New student created! Your Student ID is {student_id}.")

    else:
        # Verify existing student login
        cursor.execute('SELECT password FROM Student WHERE sid = ?', (student_id,))
        student = cursor.fetchone()
        if not student:
            print("Invalid Student ID. Exiting.")
            connection.close()
            return

        password = input("Enter your password: ").strip()
        if password != student[0]:
            print("Incorrect password. Access denied.")
            connection.close()
            return

    connection.close()

    # Student Menu Loop
    while True:
        print("\nStudent Menu:")
        print("L - List All Courses")
        print("E - Enroll in a Course")
        print("W - Withdraw from a Course")
        print("S - Search Courses")
        print("M - View My Classes")
        print("P - View Prerequisites for a Course")
        print("X - Exit")

        choice = input("Enter your choice: ").strip().upper()
        if choice == 'L':
            list_courses()
        elif choice == 'E':
            result = enroll_student(student_id)  # Capture the return value
            print(result)  # Display the feedback to the user
        elif choice == 'W':
            result = withdraw_student(student_id)  # Capture the return value
            print(result)  # Display the feedback to the user
        elif choice == 'S':
            search_courses()  # Assuming this already prints output
        elif choice == 'M':
            view_enrolled_classes(student_id)  # Assuming this already prints output
        elif choice == 'P':
            view_prerequisites()  # Assuming this already prints output
        elif choice == 'X':
            print("Exiting Student Menu.")
            break
        else:
            print("Invalid choice. Please try again.")


In [ ]:
def admin_menu():
    while True:
        print("\nAdmin Menu:")
        print("S - Manage Students (Add, Edit, Delete)")
        print("P - Manage Professors (Add, Edit, Delete)")
        print("C - Manage Courses (Add, Edit, Delete)")
        print("T - List All Professors")
        print("E - View Enrolled Students for a Course")
        print("W - Check Professor's Workload")
        print("A - Assign a Professor to a Course")
        print("LS - List All Students")  # New option
        print("X - Exit")

        choice = input("Enter your choice: ").strip().upper()
        if choice == 'S':
            # Student Management Submenu
            print("\nStudent Management:")
            print("A - Add Student")
            print("E - Edit Student")
            print("D - Delete Student")
            sub_choice = input("Enter your choice: ").strip().upper()
            if sub_choice == 'A':
                print(add_student())
            elif sub_choice == 'E':
                print(edit_student())
            elif sub_choice == 'D':
                print(delete_student())
            else:
                print("Invalid choice. Returning to Admin Menu.")
        elif choice == 'LS':  # New option for listing students
            list_students()
        elif choice == 'P':
            # Professor Management Submenu
            print("\nProfessor Management:")
            print("A - Add Professor")
            print("E - Edit Professor")
            print("D - Delete Professor")
            sub_choice = input("Enter your choice: ").strip().upper()
            if sub_choice == 'A':
                print(add_professor())
            elif sub_choice == 'E':
                print(edit_professor())
            elif sub_choice == 'D':
                print(delete_professor())
            else:
                print("Invalid choice. Returning to Admin Menu.")
        elif choice == 'C':
            # Course Management Submenu
            print("\nCourse Management:")
            print("A - Add Course")
            print("E - Edit Course")
            print("D - Delete Course")
            sub_choice = input("Enter your choice: ").strip().upper()
            if sub_choice == 'A':
                print(add_course())
            elif sub_choice == 'E':
                print(edit_course())
            elif sub_choice == 'D':
                print(delete_course())
            else:
                print("Invalid choice. Returning to Admin Menu.")
        elif choice == 'T':
            list_professors()
        elif choice == 'E':
            course_id = int(input("Enter Course ID to view enrolled students: ").strip())
            print(list_enrolled_students(course_id))
        elif choice == 'W':
            check_professor_workload()
        elif choice == 'A':
            assign_professor_to_course()
        elif choice == 'X':
            print("Exiting Admin Menu.")
            break
        else:
            print("Invalid choice. Please try again.")


In [ ]:
def main():
    create_database()
    insert_sample_data()

    while True:
        print("\nMain Menu:")
        print("1 - Student Menu")
        print("2 - Admin Menu")
        print("3 - Exit")

        choice = input("Enter your choice: ").strip()
        if choice == '1':
            student_menu()
        elif choice == '2':
            admin_menu()
        elif choice == '3':
            print("Exiting Program.")
            break
        else:
            print("Invalid choice. Please try again.")

if __name__ == "__main__":
    main()



Main Menu:
1 - Student Menu
2 - Admin Menu
3 - Exit
Enter your choice: 1
Enter your Student ID (-1 to create a new student): 1
Enter your password: password1

Student Menu:
L - List All Courses
E - Enroll in a Course
W - Withdraw from a Course
S - Search Courses
M - View My Classes
P - View Prerequisites for a Course
X - Exit
Enter your choice: x
Exiting Student Menu.

Main Menu:
1 - Student Menu
2 - Admin Menu
3 - Exit
Enter your choice: 1
Enter your Student ID (-1 to create a new student): 1
Enter your password: password1

Student Menu:
L - List All Courses
E - Enroll in a Course
W - Withdraw from a Course
S - Search Courses
M - View My Classes
P - View Prerequisites for a Course
X - Exit
Enter your choice: l

Courses Available:
Course ID: 1, Name: Intro to CS, Credits: 3, Description: Introduction to programming, Capacity: 25
  Professor: Dr. Michelle Zhu, Day: Monday, Time: 10:00 - 11:30, Location: Room 101
Course ID: 1, Name: Intro to CS, Credits: 3, Description: Introduction to 